<a id='report-production'></a>
<p style="font-weight:bold;"> <span style="font-size: 36px"> Reports</span> </p>

For demonstration purposes we import here data for some *Group of Insurance Contract* (GIC) and *Group of Reinsurance Contract* (GRIC) - the import is triggered in the [Set up data and configuration](#set-up-data-and-configuration) section.
<br> The imported data set consists of cash flows, actuals, and parameters.
<br> Input files can be found in the **File** directory. You are invited to change them or upload your own or add new data in the [CloseImportTemplate](../Import/CloseImportTemplate) notebook. 
<br> For simplicity, we import similar transactional data for all GICs and GRICs. Each *Group of Contracts* produces different figures due to differences in parameters such as *Liability Type*, *Oci type* or *Premium allocation factor* to Contractual Service Margin.

Follow the instructions below for a guided interaction with the reports.

# Set up data and configuration

Choose to run the Reports notebook either with the set of Systemorph data in memory or with the data present in the Database: 
- #!eval-notebook "../Database/Configure" : connects to a physical Database
- #!eval-notebook "../Import/CloseImportTemplate" : uses the in-memory set up

We use here the in-memory set up.

In [0]:
#!eval-notebook "../Import/CloseImportTemplate"

In [0]:
Workspace.InitializeFrom(DataSource);
ifrs17.Reset(Workspace)

# Best Estimate

Present values of the [best-estimate](https://portal.systemorph.cloud/project/ifrs17/env/v1.1.0/Report/ReportScopes#best-estimate) future cash flows are shown here in an Analysis of Change report.

The granularity of the reported figures can be modified by changing the Column Slices options.
For example one can add <code>"GroupOfContract"</code> to separate the contributions of the individual Group of Contracts.

Aggregated values are displayed when the data has a finer granularity than the one selected by the report slice options.

In [0]:
var pv = ifrs17.PresentValues;
pv.ReportingNode = "CH";
pv.ReportingPeriod = (2021, 3);
pv.CurrencyType = CurrencyType.Contractual;
pv.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
pv.DataFilter = new [] {("GroupOfContract", "DT1.1")};
(await pv.ToReportAsync) with {Height = 720}

# Risk Adjustment

Present values of the [risk adjustment](https://portal.systemorph.cloud/project/ifrs17/env/v1.1.0/Report/ReportScopes#risk-adjustment) future cash flows are shown here.

The additional ColumnSlices are added to the view as the inner column. This can dicrease the readability of the report. For example, adding the slice by "GroupOfContract" leaves the lock-in and current rate contributions far apart and difficult to compare. The re-order of default slices with custom slices is achieved by esplicitly add the default slice among the custom slices. In our case, you can try entering both "GroupOfContract" and "EconomicBasis" separated by a comma in the ColumnSlices.

In [0]:
var ra = ifrs17.RiskAdjustments;
ra.ReportingNode = "CH";
ra.ReportingPeriod = (2021, 3);
ra.ColumnSlices = new string[]{};//"GroupOfContract", "EconomicBasis"
ra.DataFilter = null;//new [] {("GroupOfContract", "DT1.2")};
(await ra.ToReportAsync) with {Height = 800}

# Written Actuals

[Written Actuals](https://portal.systemorph.cloud/project/ifrs17/env/v1.1.0/Report/ReportScopes#written-accrual-deferral) are shown here. 

In this case, the analysis of change view is replaced with a default slice by the **AmountTypes**. Only the amount type with non zero value are displayed. 
<br>Filters can be applied to reports in order to isolate a sub-set of the data. They are specified by the name of the dimension to filter and the system name of the desired value. For example, to investigate the contribution of a single Group of Contract the following filter can be applied: new [] {("GroupOfContract", "DT1.1")}.

In [0]:
var writtenActual = ifrs17.WrittenActuals;
writtenActual.ReportingNode = "CH";
writtenActual.ReportingPeriod = (2021, 3);
writtenActual.ColumnSlices = new string[]{};//"GroupOfContract"
writtenActual.DataFilter =  null; //new [] {("GroupOfContract", "DT1.1")};
(await writtenActual.ToReportAsync) with {Height = 400}

## Advance, Overdue Actuals

Actuals payed in [Advance](https://portal.systemorph.cloud/project/ifrs17/env/v1.1.0/Report/ReportScopes#written-accrual-deferral)
or [Overdue](https://portal.systemorph.cloud/project/ifrs17/env/v1.1.0/Report/ReportScopes#written-accrual-deferral) are shown here together in a simplified Analysis of Change. 

Combining Filters and ColumnSlices facilitate the report analysis. For example, you can select "GroupOfContract" as column slices with a Filter on EstimateType "AA" to analyse the Advance Actuals for all Group of Contract. 

In [0]:
var accrualActual = ifrs17.AccrualActuals;
accrualActual.ReportingNode = "CH";
accrualActual.ReportingPeriod = (2021, 3);
accrualActual.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
accrualActual.DataFilter = null; //new [] {("EstimateType", "AA")};
(await accrualActual.ToReportAsync) with {Height = 400}

## Deferrable Actuals

[Deferrable Actuals](https://portal.systemorph.cloud/project/ifrs17/env/v1.1.0/Report/ReportScopes#written-accrual-deferral) are shown here. Amortization of the deferrable amount is computed using the Coverage Unit pattern.  

In [0]:
var deferrableActual = ifrs17.DeferralActuals;
deferrableActual.ReportingNode = "CH";
deferrableActual.ReportingPeriod = (2021, 3);
deferrableActual.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
deferrableActual.DataFilter = null;//new [] {("GroupOfContract", "DT1.1")};
(await deferrableActual.ToReportAsync) with {Height = 400}

# Fulfilment Cash flow

Present Value of the [Fulfilment Cash flow](https://portal.systemorph.cloud/project/ifrs17/env/v1.1.0/Report/ReportScopes#fulfillment-cash-flows) are shown here. 
<br> The individual contributions from Best Estimate and Risk Adjustment can be visualized slicing by **EstimateType**.

Filters can be applied to report to isolate a sub-set of the data. For example you can Filter by a specific Group of Contract using its system name using : new [] {("GroupOfContract", "DT1.1")}.


In [0]:
var fulfillmentCashflows = ifrs17.FulfillmentCashflows;
fulfillmentCashflows.ReportingNode = "CH";
fulfillmentCashflows.ReportingPeriod = (2021, 3);
fulfillmentCashflows.ColumnSlices = new string[]{};//"EstimateType"
fulfillmentCashflows.DataFilter = null;// new [] {("GroupOfContract", "DT1.1")};
(await fulfillmentCashflows.ToReportAsync) with {Height = 750}

# Actuarial Experience Adjustment

A comparison between [Written Actual](#written-actual) and the Releases of the [Best Estimate](#present-value) is reported in the [Actuarial Experience Adjustment](https://portal.systemorph.cloud/project/ifrs17/env/v1.1.0/Report/ReportScopes#experience-adjustment).

In [0]:
var experienceAdjustments = ifrs17.ExperienceAdjustments;
experienceAdjustments.ReportingNode = "CH";
experienceAdjustments.ReportingPeriod = (2021, 3);
experienceAdjustments.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
experienceAdjustments.DataFilter = null; //new [] {("GroupOfContract", "DT1.1")};
(await experienceAdjustments.ToReportAsync) with {Height = 300}

# LRC Technical Margin

In the [Technical Margin](https://portal.systemorph.cloud/project/ifrs17/env/v1.1.0/Report/ReportScopes#technical-margin) report we present a unified view on the figures that are allocated to either Contractual Service Margin or to Loss Component. 
<br> The Analysis of Change is expanded with few more steps such as **Experience Adjustment** and **Amortization**.

In [0]:
var technicalMargins = ifrs17.TechnicalMargins;
technicalMargins.ReportingNode = "CH";
technicalMargins.ReportingPeriod = (2021, 3);
technicalMargins.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
technicalMargins.DataFilter = null; //new [] {("GroupOfContract", "DT1.1")};
(await technicalMargins.ToReportAsync) with {Height = 600}

# Contractual Service Margin / Loss Component / Loss Recovery Component

The Contractual Service Margin (CSM) / Loss Component (LC) / Loss Recovery Component (LR) [report](https://portal.systemorph.cloud/project/ifrs17/env/v1.1.0/Report/ReportScopes#technical-margin) are here shown side by side as the allocation to profit or loss is done at each step of the Analysis of Change. 

A default slice by EstimateType - which distinguish between CSM, LC and LR contributions - is automatically enforced. 

In [0]:
var allocatedTechnicalMargins = ifrs17.AllocatedTechnicalMargins;
allocatedTechnicalMargins.ReportingNode = "CH";
allocatedTechnicalMargins.ReportingPeriod = (2021, 3);
allocatedTechnicalMargins.ColumnSlices = new string[]{};//"GroupOfContract", "EstimateType"
allocatedTechnicalMargins.DataFilter = null; //new [] {("GroupOfContract", "DT1.1")};
(await allocatedTechnicalMargins.ToReportAsync) with {Height = 700}

# LRC Actuarial

The [Actuarial Liability for Remaining Coverage](https://portal.systemorph.cloud/project/ifrs17/env/v1.1.0/Report/ReportScopes#lrc-actuarial-actuarial-liability-for-remaining-coverage) report shows figures from Fulfilment Cash flow discounted with current yield curve, and the allocated techinical margin. 

In [0]:
var actuarialLrc = ifrs17.ActuarialLrc;
actuarialLrc.ReportingNode = "CH";
actuarialLrc.ReportingPeriod = (2021, 3);
actuarialLrc.ColumnSlices = new string[]{};//"GroupOfContract"
actuarialLrc.DataFilter = null; //new [] {("GroupOfContract", "DT1.1")};
(await actuarialLrc.ToReportAsync) with {Height = 750}

# LRC

The [Liability for Remaining Coverage](https://portal.systemorph.cloud/project/ifrs17/env/v1.1.0/Report/ReportScopes#lrc-liability-for-remaining-coverage) report adds to the [Actuarial Liability for Remaining Coverage](#lrc-actuarial) the contribution of and accrual actuals. A simplified AoC Chain is used to allow comparison of the balance change between Actuals and Present Values. 

In [0]:
var lrc = ifrs17.Lrc;
lrc.ReportingNode = "CH";
lrc.ReportingPeriod = (2021, 3);
lrc.ColumnSlices = new string[]{};//"GroupOfContract",
lrc.DataFilter = null; //new [] {("GroupOfContract", "DT1.1")};
(await lrc.ToReportAsync) with {Height = 250}

# LIC Actuarial

The [Actuarial Liability of Incurred Claims](https://portal.systemorph.cloud/project/ifrs17/env/v1.1.0/Report/ReportScopes#lic-actuarial-actuarial-liability-for-incurred-claims) report shows figures from Fulfilment Cash flow discounted with current yield curve. 

In [0]:
var actuarialLic = ifrs17.ActuarialLic;
actuarialLic.ReportingNode = "CH";
actuarialLic.ReportingPeriod = (2021, 3);
actuarialLic.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
actuarialLic.DataFilter = null; //new [] {("GroupOfContract", "DT1.1")};
(await actuarialLic.ToReportAsync) with {Height = 750}

# LIC

The [Liability for Incurred Claims](https://portal.systemorph.cloud/project/ifrs17/env/v1.1.0/Report/ReportScopes#lic-liability-for-incurred-claims) adds to the [Actuarial Liability for Incurred Claims](#lic-actuarial) the contribution of and accrual actuals.

In [0]:
var lic = ifrs17.Lic;
lic.ReportingNode = "CH";
lic.ReportingPeriod = (2021, 3);
lic.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
lic.DataFilter = null; //new [] {("GroupOfContract", "DT1.1")};
(await lic.ToReportAsync) with {Height = 250}

# Financial Performance

The [Financial Performance](https://portal.systemorph.cloud/project/ifrs17/env/v1.1.0/Report/ReportScopes#ifrs-17-financial-performance) report discloses the Change in Estimate of the IFRS 17 balance sheet items ([LRC](#lrc) and [LIC](#lic)) and the relevant incurred cash flows (Premiums, Claims, Expenses, etc...) for the given period.

Use the expand and collapse buttons in the report rows to change the granularity of the figures displayed.

In [0]:
var financialPerformance = ifrs17.FinancialPerformance;
financialPerformance.ReportingNode = "CH";
financialPerformance.ReportingPeriod = (2021, 3);
financialPerformance.ColumnSlices = new string[]{};//"GroupOfContract"
financialPerformance.DataFilter = null; //new [] {("GroupOfContract", "DT1.1")};
(await financialPerformance.ToReportAsync) with { Height = 900, GroupDefaultExpanded = 3}